In [1]:
import requests
from requests.models import Response
import logging
import json
import time
from pathlib import Path


class AzureContentUnderstandingClient:
    def __init__(
        self,
        endpoint: str,
        api_version: str,
        subscription_key: str = None,
        token_provider: callable = None,
        x_ms_useragent: str = "cu-sample-code",
    ):
        if not subscription_key and not token_provider:
            raise ValueError(
                "Either subscription key or token provider must be provided."
            )
        if not api_version:
            raise ValueError("API version must be provided.")
        if not endpoint:
            raise ValueError("Endpoint must be provided.")

        self._endpoint = endpoint.rstrip("/")
        self._api_version = api_version
        self._logger = logging.getLogger(__name__)
        self._headers = self._get_headers(
            subscription_key, token_provider(), x_ms_useragent
        )

    def _get_analyzer_url(self, endpoint, api_version, analyzer_id):
        return f"{endpoint}/contentunderstanding/analyzers/{analyzer_id}?api-version={api_version}"  # noqa

    def _get_analyzer_list_url(self, endpoint, api_version):
        return f"{endpoint}/contentunderstanding/analyzers?api-version={api_version}"

    def _get_analyze_url(self, endpoint, api_version, analyzer_id):
        return f"{endpoint}/contentunderstanding/analyzers/{analyzer_id}:analyze?api-version={api_version}"  # noqa

    def _get_training_data_config(
        self, storage_container_sas_url, storage_container_path_prefix
    ):
        return {
            "containerUrl": storage_container_sas_url,
            "kind": "blob",
            "prefix": storage_container_path_prefix,
        }

    def _get_headers(self, subscription_key, api_token, x_ms_useragent):
        """Returns the headers for the HTTP requests.
        Args:
            subscription_key (str): The subscription key for the service.
            api_token (str): The API token for the service.
            enable_face_identification (bool): A flag to enable face identification.
        Returns:
            dict: A dictionary containing the headers for the HTTP requests.
        """
        headers = (
            {"Ocp-Apim-Subscription-Key": subscription_key}
            if subscription_key
            else {"Authorization": f"Bearer {api_token}"}
        )
        headers["x-ms-useragent"] = x_ms_useragent
        return headers

    def get_all_analyzers(self):
        """
        Retrieves a list of all available analyzers from the content understanding service.

        This method sends a GET request to the service endpoint to fetch the list of analyzers.
        It raises an HTTPError if the request fails.

        Returns:
            dict: A dictionary containing the JSON response from the service, which includes
                  the list of available analyzers.

        Raises:
            requests.exceptions.HTTPError: If the HTTP request returned an unsuccessful status code.
        """
        response = requests.get(
            url=self._get_analyzer_list_url(self._endpoint, self._api_version),
            headers=self._headers,
        )
        response.raise_for_status()
        return response.json()

    def get_analyzer_detail_by_id(self, analyzer_id):
        """
        Retrieves a specific analyzer detail through analyzerid from the content understanding service.
        This method sends a GET request to the service endpoint to get the analyzer detail.

        Args:
            analyzer_id (str): The unique identifier for the analyzer.

        Returns:
            dict: A dictionary containing the JSON response from the service, which includes the target analyzer detail.

        Raises:
            HTTPError: If the request fails.
        """
        response = requests.get(
            url=self._get_analyzer_url(self._endpoint, self._api_version, analyzer_id),
            headers=self._headers,
        )
        response.raise_for_status()
        return response.json()

    def begin_create_analyzer(
        self,
        analyzer_id: str,
        analyzer_template: dict = None,
        analyzer_template_path: str = "",
        training_storage_container_sas_url: str = "",
        training_storage_container_path_prefix: str = "",
    ):
        """
        Initiates the creation of an analyzer with the given ID and schema.

        Args:
            analyzer_id (str): The unique identifier for the analyzer.
            analyzer_template (dict, optional): The schema definition for the analyzer. Defaults to None.
            analyzer_template_path (str, optional): The file path to the analyzer schema JSON file. Defaults to "".
            training_storage_container_sas_url (str, optional): The SAS URL for the training storage container. Defaults to "".
            training_storage_container_path_prefix (str, optional): The path prefix within the training storage container. Defaults to "".

        Raises:
            ValueError: If neither `analyzer_template` nor `analyzer_template_path` is provided.
            requests.exceptions.HTTPError: If the HTTP request to create the analyzer fails.

        Returns:
            requests.Response: The response object from the HTTP request.
        """
        if analyzer_template_path and Path(analyzer_template_path).exists():
            with open(analyzer_template_path, "r") as file:
                analyzer_template = json.load(file)

        if not analyzer_template:
            raise ValueError("Analyzer schema must be provided.")

        if (
            training_storage_container_sas_url
            and training_storage_container_path_prefix
        ):  # noqa
            analyzer_template["trainingData"] = self._get_training_data_config(
                training_storage_container_sas_url,
                training_storage_container_path_prefix,
            )

        headers = {"Content-Type": "application/json"}
        headers.update(self._headers)

        response = requests.put(
            url=self._get_analyzer_url(self._endpoint, self._api_version, analyzer_id),
            headers=headers,
            json=analyzer_template,
        )
        response.raise_for_status()
        self._logger.info(f"Analyzer {analyzer_id} create request accepted.")
        return response

    def delete_analyzer(self, analyzer_id: str):
        """
        Deletes an analyzer with the specified analyzer ID.

        Args:
            analyzer_id (str): The ID of the analyzer to be deleted.

        Returns:
            response: The response object from the delete request.

        Raises:
            HTTPError: If the delete request fails.
        """
        response = requests.delete(
            url=self._get_analyzer_url(self._endpoint, self._api_version, analyzer_id),
            headers=self._headers,
        )
        response.raise_for_status()
        self._logger.info(f"Analyzer {analyzer_id} deleted.")
        return response

    def begin_analyze(self, analyzer_id: str, file_location: str, file_data):
        """
        Begins the analysis of a file or URL using the specified analyzer.

        Args:
            analyzer_id (str): The ID of the analyzer to use.
            file_location (str): The path to the file or the URL to analyze.

        Returns:
            Response: The response from the analysis request.

        Raises:
            ValueError: If the file location is not a valid path or URL.
            HTTPError: If the HTTP request returned an unsuccessful status code.
        """
        
        if file_data:
            data = file_data
            headers = {"Content-Type": "application/octet-stream"}
        else:  
            data = None
            if Path(file_location).exists():
                with open(file_location, "rb") as file:
                    data = file.read()
                headers = {"Content-Type": "application/octet-stream"}
            elif "https://" in file_location or "http://" in file_location:
                data = {"url": file_location}
                headers = {"Content-Type": "application/json"}
            else:
                raise ValueError("File location must be a valid path or URL.")

        headers.update(self._headers)
        if isinstance(data, dict):
            response = requests.post(
                url=self._get_analyze_url(
                    self._endpoint, self._api_version, analyzer_id
                ),
                headers=headers,
                json=data,
            )
        else:
            response = requests.post(
                url=self._get_analyze_url(
                    self._endpoint, self._api_version, analyzer_id
                ),
                headers=headers,
                data=data,
            )
        print("rs.....",response)
        response.raise_for_status()
        self._logger.info(
            f"Analyzing file {file_location} with analyzer: {analyzer_id}"
        )
        return response

    def get_image_from_analyze_operation(
        self, analyze_response: Response, image_id: str
    ):
        """Retrieves an image from the analyze operation using the image ID.
        Args:
            analyze_response (Response): The response object from the analyze operation.
            image_id (str): The ID of the image to retrieve.
        Returns:
            bytes: The image content as a byte string.
        """
        operation_location = analyze_response.headers.get("operation-location", "")
        if not operation_location:
            raise ValueError(
                "Operation location not found in the analyzer response header."
            )
        operation_location = operation_location.split("?api-version")[0]
        image_retrieval_url = (
            f"{operation_location}/images/{image_id}?api-version={self._api_version}"
        )
        try:
            response = requests.get(url=image_retrieval_url, headers=self._headers)
            response.raise_for_status()

            assert response.headers.get("Content-Type") == "image/jpeg"

            return response.content
        except requests.exceptions.RequestException as e:
            print(f"HTTP request failed: {e}")
            return None

    def poll_result(
        self,
        response: Response,
        timeout_seconds: int = 120,
        polling_interval_seconds: int = 2,
    ):
        """
        Polls the result of an asynchronous operation until it completes or times out.

        Args:
            response (Response): The initial response object containing the operation location.
            timeout_seconds (int, optional): The maximum number of seconds to wait for the operation to complete. Defaults to 120.
            polling_interval_seconds (int, optional): The number of seconds to wait between polling attempts. Defaults to 2.

        Raises:
            ValueError: If the operation location is not found in the response headers.
            TimeoutError: If the operation does not complete within the specified timeout.
            RuntimeError: If the operation fails.

        Returns:
            dict: The JSON response of the completed operation if it succeeds.
        """
        operation_location = response.headers.get("operation-location", "")
        if not operation_location:
            raise ValueError("Operation location not found in response headers.")

        headers = {"Content-Type": "application/json"}
        headers.update(self._headers)

        start_time = time.time()
        while True:
            elapsed_time = time.time() - start_time
            if elapsed_time > timeout_seconds:
                raise TimeoutError(
                    f"Operation timed out after {timeout_seconds:.2f} seconds."
                )

            response = requests.get(operation_location, headers=self._headers)
            response.raise_for_status()
            status = response.json().get("status").lower()
            if status == "succeeded":
                self._logger.info(
                    f"Request result is ready after {elapsed_time:.2f} seconds."
                )
                return response.json()
            elif status == "failed":
                self._logger.error(f"Request failed. Reason: {response.json()}")
                raise RuntimeError("Request failed.")
            else:
                self._logger.info(
                    f"Request {operation_location.split('/')[-1].split('?')[0]} in progress ..."
                )
            time.sleep(polling_interval_seconds)

In [3]:
# Import required modules
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential, AzureCliCredential
import sys
from pathlib import Path
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
# from content_understanding_client import AzureContentUnderstandingClient
from pathlib import Path
from azure.storage.filedatalake import DataLakeServiceClient
from datetime import datetime, timedelta
from openai import AzureOpenAI
import re
import time
import struct
import pyodbc
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

key_vault_name = "kmluktljhccz7p-kv"
managed_identity_client_id = "e791bf0e-4c05-4db6-bc80-ccfa288be2a7"

file_system_client_name = "data"
directory_name = "videodata"

# Set the name of the Azure Key Vault
credential = DefaultAzureCredential()
token = credential.get_token("https://vault.azure.net/.default")

def get_secrets_from_kv(kv_name, secret_name):
    # Create a secret client object using the credential and Key Vault name
    secret_client = SecretClient(
        vault_url=f"https://{kv_name}.vault.azure.net/", credential=credential
    )

    # Retrieve the secret value
    return secret_client.get_secret(secret_name).value


# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))

AZURE_AI_ENDPOINT = get_secrets_from_kv(key_vault_name, "AZURE-OPENAI-CU-ENDPOINT")
AZURE_OPENAI_CU_KEY = get_secrets_from_kv(key_vault_name, "AZURE-OPENAI-CU-KEY")
AZURE_AI_API_VERSION = "2024-12-01-preview"

search_endpoint = get_secrets_from_kv(key_vault_name, "AZURE-SEARCH-ENDPOINT")
search_key = get_secrets_from_kv(key_vault_name, "AZURE-SEARCH-KEY")

openai_api_key = get_secrets_from_kv(key_vault_name, "AZURE-OPENAI-KEY")
openai_api_base = get_secrets_from_kv(key_vault_name, "AZURE-OPENAI-ENDPOINT")
openai_api_version = get_secrets_from_kv(
    key_vault_name, "AZURE-OPENAI-PREVIEW-API-VERSION"
)
deployment = get_secrets_from_kv(key_vault_name, "AZURE-OPEN-AI-DEPLOYMENT-MODEL")

token_provider = get_bearer_token_provider(
    credential, "https://cognitiveservices.azure.com/.default"
)
client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    subscription_key=AZURE_OPENAI_CU_KEY,
    token_provider=token_provider,
)

ANALYZER_ID = "ckm-video"
ANALYZER_TEMPLATE_FILE = "C:\Projects\Microsoft\Conversation-Knowledge-Mining-Solution-Accelerator\infra\data\ckm-analyzer_config_video.json"

# Create analyzer
# response = client.begin_create_analyzer(
#     ANALYZER_ID, analyzer_template_path=ANALYZER_TEMPLATE_FILE
# )
# result = client.poll_result(response)
# print(result)
account_name = get_secrets_from_kv(key_vault_name, "ADLS-ACCOUNT-NAME")
account_url = f"https://{account_name}.dfs.core.windows.net"

In [5]:
service_client = DataLakeServiceClient(
    account_url, credential=credential, api_version="2023-01-03"
)

# Function: Get Embeddings
def get_embeddings(text, openai_api_base, openai_api_version, openai_api_key):
    model_id = "text-embedding-ada-002"
    client = AzureOpenAI(
        api_version=openai_api_version,
        azure_endpoint=openai_api_base,
        api_key=openai_api_key,
    )

    embedding = client.embeddings.create(input=text, model=model_id).data[0].embedding

    return embedding

# Function: Clean Spaces with Regex -
def clean_spaces_with_regex(text):
    # Use a regular expression to replace multiple spaces with a single space
    cleaned_text = re.sub(r"\s+", " ", text)
    # Use a regular expression to replace consecutive dots with a single dot
    cleaned_text = re.sub(r"\.{2,}", ".", cleaned_text)
    return cleaned_text

def chunk_data(text):
    tokens_per_chunk = 1024
    text = clean_spaces_with_regex(text)

    sentences = text.split(". ")  # Split text into sentences
    chunks = []
    current_chunk = ""
    current_chunk_token_count = 0

    # Iterate through each sentence
    for sentence in sentences:
        # Split sentence into tokens
        tokens = sentence.split()

        # Check if adding the current sentence exceeds tokens_per_chunk
        if current_chunk_token_count + len(tokens) <= tokens_per_chunk:
            # Add the sentence to the current chunk
            if current_chunk:
                current_chunk += ". " + sentence
            else:
                current_chunk += sentence
            current_chunk_token_count += len(tokens)
        else:
            # Add current chunk to chunks list and start a new chunk
            chunks.append(current_chunk)
            current_chunk = sentence
            current_chunk_token_count = len(tokens)

    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def prepare_search_doc(content, document_id):
    chunks = chunk_data(content)
    chunk_num = 0
    for chunk in chunks:
        chunk_num += 1
        chunk_id = document_id + "_" + str(chunk_num).zfill(2)

        try:
            v_contentVector = get_embeddings(
                str(chunk), openai_api_base, openai_api_version, openai_api_key
            )
        except Exception:
            time.sleep(30)
            try:
                v_contentVector = get_embeddings(
                    str(chunk), openai_api_base, openai_api_version, openai_api_key
                )
            except Exception:
                v_contentVector = []
        result = {
            "id": chunk_id,
            "chunk_id": chunk_id,
            "content": chunk,
            "sourceurl": path.name.split("/")[-1],
            "contentVector": v_contentVector,
        }
    return result

index_name = "call_transcripts_index"

search_credential = AzureKeyCredential(search_key)
search_client = SearchClient(search_endpoint, index_name, search_credential)

driver = "{ODBC Driver 18 for SQL Server}"
server = get_secrets_from_kv(key_vault_name, "SQLDB-SERVER")
database = get_secrets_from_kv(key_vault_name, "SQLDB-DATABASE")

token_bytes = credential.get_token(
    "https://database.windows.net/.default"
).token.encode("utf-16-LE")
token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256

# Set up the connection
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};"

conn = pyodbc.connect(
    connection_string, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct}
)

# conn = pymssql.connect(server, username, password, database)
cursor = conn.cursor()
print("Connected to the database")
cursor.execute("DROP TABLE IF EXISTS vprocessed_data")
conn.commit()

create_processed_data_sql = """CREATE TABLE vprocessed_data (
                ConversationId varchar(255) NOT NULL PRIMARY KEY,
                EndTime varchar(255),
                StartTime varchar(255),
                Content varchar(max),
                summary varchar(3000),
                satisfied varchar(255),
                sentiment varchar(255),
                topic varchar(255),
                key_phrases nvarchar(max),
                complaint varchar(255), 
                mined_topic varchar(255)
            );"""
cursor.execute(create_processed_data_sql)
conn.commit()

cursor.execute('DROP TABLE IF EXISTS vprocessed_data_key_phrases')
conn.commit()

create_processed_data_sql = """CREATE TABLE vprocessed_data_key_phrases (
                    ConversationId varchar(255),
                    key_phrase varchar(500), 
                    sentiment varchar(255),
                    topic varchar(255), 
                    StartTime varchar(255),
                    );"""
cursor.execute(create_processed_data_sql)
conn.commit()

file_system_client = service_client.get_file_system_client(file_system_client_name)
paths = file_system_client.get_paths(path=directory_name)

conversationIds = []
docs = []
counter = 0
# process and upload audio files to search index
for path in paths:
    file_client = file_system_client.get_file_client(path.name)
    data_file = file_client.download_file()
    data = data_file.readall()
    try:
        # Analyzer file
        ANALYZER_ID = "ckm-video"
        response = client.begin_analyze(ANALYZER_ID, file_location="", file_data=data)
        print(">>>>>res", response)
        result = client.poll_result(response)
        print(".......",result)
        file_name = path.name.split("/")[-1]
        start_time = file_name.replace(".mp4", "")[-19:]

        timestamp_format = "%Y-%m-%d %H_%M_%S"  # Adjust format if necessary
        start_timestamp = datetime.strptime(start_time, timestamp_format)

        conversation_id = result["id"]
        conversationIds.append(result["id"])
        print(">>>>>>>>>>>>>>>")
        duration = int(
            result["result"]["contents"][0]["fields"]["Duration"]["valueString"]
        )
        end_timestamp = str(start_timestamp + timedelta(seconds=duration))
        end_timestamp = end_timestamp.split(".")[0]

        summary = result["result"]["contents"][0]["fields"]["summary"]["valueString"]
        satisfied = result["result"]["contents"][0]["fields"]["satisfied"].get("valueString")
        sentiment = result["result"]["contents"][0]["fields"]["sentiment"][
            "valueString"
        ]
        topic = result["result"]["contents"][0]["fields"]["topic"]["valueString"]
        key_phrases = result["result"]["contents"][0]["fields"]["keyPhrases"][
            "valueString"
        ]
        print(">>>>>>>///...",key_phrases)
        complaint = result["result"]["contents"][0]["fields"]["complaint"].get("valueString")
        content = result["result"]["contents"][0]["fields"]["content"]["valueString"]
        # print(topic)
        cursor.execute(
            f"INSERT INTO vprocessed_data (ConversationId, EndTime, StartTime, Content, summary, satisfied, sentiment, topic, key_phrases, complaint) VALUES (?,?,?,?,?,?,?,?,?,?)",
            (
                conversation_id,
                end_timestamp,
                start_timestamp,
                content,
                summary,
                satisfied,
                sentiment,
                topic,
                key_phrases,
                complaint,
            ),
        )
        conn.commit()

        keyPhrases = key_phrases.split(",")
        for keyPhrase in keyPhrases:
            cursor.execute(
                f"INSERT INTO vprocessed_data_key_phrases (ConversationId, key_phrase, sentiment) VALUES (?,?,?)",
                (conversation_id, keyPhrase, sentiment),
            )

        document_id = conversation_id
        # print(content)
        print(">>>>>>,,,,,,", document_id)
        result = prepare_search_doc(content, document_id)
        docs.append(result)
        counter += 1
    except Exception as e:
        print(">>>>>>>",str(e))
    print(">>>>//",docs)
    if docs != [] and counter % 10 == 0:
        result = search_client.upload_documents(documents=docs)
        docs = []

# upload the last batch
if docs != []:
    search_client.upload_documents(documents=docs)

conn.commit()
cursor.close()
conn.close()

Connected to the database
rs..... <Response [202]>
>>>>>res <Response [202]>
....... {'id': 'fce7039a-8eff-4df1-912b-76998d8f1b92', 'status': 'Succeeded', 'result': {'analyzerId': 'ckm-video', 'apiVersion': '2024-12-01-preview', 'createdAt': '2025-04-18T11:03:58Z', 'warnings': [], 'contents': [{'markdown': "```WEBVTT\n\n00:01.400 --> 00:06.560\n<v Speaker 1>When it comes to the neural TTS, in order to get a good voice, it's better to have good data.\n\n00:07.600 --> 00:13.320\n<v Speaker 2>To achieve that, we build a universal TTS model based on 3,000 hours of data.\n\n00:13.440 --> 00:23.640\n<v Speaker 1>We actually accumulated tons of the data so that this universal model is able to capture the nuance of the audio and generate a more natural voice for the algorithm.\n\n00:24.080 --> 00:29.120\n<v Speaker 3>What we liked about cognitive services offerings were that they had a much higher fidelity.\n\n00:29.600 --> 00:32.880\n<v Speaker 3>And they sounded a lot more like an actual hum

In [16]:
print("test")

test
